# Bibliotecas

In [1]:
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import funcoes_bert as fb

import time

## Constantes

In [2]:
SEED = 42

# Lendo os Arquivos

In [3]:
#arquivos = ['ale_1_1']
arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
#arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df)

    df = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_val.append(df)

    df = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_teste.append(df)

In [4]:
#lista_df_val[0].dtypes

In [5]:
#print(f"Tamanho do Dataset de Treino 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_treino[0].shape[0]}\t| {lista_df_treino[2].shape[0]}\t|\n5 por 1\t| {lista_df_treino[1].shape[0]}\t| {lista_df_treino[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Validação 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_val[0].shape[0]}\t| {lista_df_val[2].shape[0]}\t|\n5 por 1\t| {lista_df_val[1].shape[0]}\t| {lista_df_val[3].shape[0]}\t|")
#print(f"\nTamanho do Dataset de Teste 1 por 1:\n\n\t| Ale\t| Hard\t|\n1 por 1\t| {lista_df_teste[0].shape[0]}\t| {lista_df_teste[2].shape[0]}\t|\n5 por 1\t| {lista_df_teste[1].shape[0]}\t| {lista_df_teste[3].shape[0]}\t|")


# BERT

In [6]:
def salvar_modelo_bert(modelo, nome):
    
    nome_arquivo = f"Dados/Modelos/BERT/{nome}/"
    modelo.save_pretrained(nome_arquivo)

## Rodando o BERT

In [7]:
lista_df_resultado = []
tam = 20
for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):
    
    X_treino = df_treino[["titulo_1", "titulo_2"]]
    X_val = df_val[["titulo_1", "titulo_2"]]
    X_teste = df_teste[["titulo_1", "titulo_2"]]

    y_treino = df_treino["match"].to_list()
    y_val = df_val["match"].to_list()
    y_teste = df_teste["match"].to_list()

    inicio_tempo = time.time()

    #y_teste = y_teste[:tam]
    #(modelo, historico, y_pred) = fb.pipeline_bert(X_treino[:tam], y_treino[:tam], X_val[:tam], y_val[:tam], X_teste[:tam], y_teste)
    (modelo, historico, y_pred) = fb.pipeline_bert(X_treino, y_treino, X_val, y_val, X_teste, y_teste)
    
    final_tempo = time.time()

    tempo = final_tempo - inicio_tempo

    pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/BERT/Histórico/{nome}_historico.csv', index = False)

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_resultado = pd.DataFrame(relatorio).transpose()
    df_resultado['modelo'] = nome
    df_resultado['tempo'] = tempo

    df_resultado.to_csv(f'Dados/Resultados/BERT/Relatório/{nome}_relatório.csv', index = True)

    df_y = pd.DataFrame(
                        list(zip(
                                 y_teste, y_pred,
                                 df_teste["categoria"].to_list(),
                                 df_teste["titulo_1"].to_list(),
                                 df_teste["titulo_2"].to_list()
                                )
                       ), columns = ['match', 'pred', 'categoria', 'titulo_1', 'titulo_2'])
    df_y.to_csv(f'Dados/Resultados/BERT/Resultado/{nome}_y.csv', index = False)

    salvar_modelo_bert(modelo, nome)

    break

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
6511/6511 [==============================] - 1704s 259ms/step - loss: 0.1170 - accuracy: 0.9590 - val_loss: 0.0569 - val_accuracy: 0.9778
Epoch 2/3
6511/6511 [==============================] - 1691s 260ms/step - loss: 0.0602 - accuracy: 0.9791 - val_loss: 0.0752 - val_accuracy: 0.9756
Epoch 3/3
6511/6511 [==============================] - 1691s 260ms/step - loss: 0.0541 - accuracy: 0.9840 - val_loss: 0.0777 - val_accuracy: 0.9749


In [8]:
#print(classification_report(y_teste, y_pred))

In [ ]:
    a

In [ ]:
'''print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))'''

In [ ]:
#lista_df_resultado

# Juntando os Relatórios

### Carregando e Concatenando

Lê os arquivos com os resultados e depois concatena em um único dataframe

In [12]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
lista_df_resultados = []
for nome in arquivos:

    df = pd.read_csv(f"Dados/Resultados/BERT/Relatório/{nome}_relatório.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_resultados.append(df)

df_resultados = pd.concat(lista_df_resultados, ignore_index = False)

In [13]:
df_resultados.rename(columns = {'Unnamed: 0':'info'}, inplace = True)
df_resultados

,info,precision,recall,f1-score,support,modelo,tempo
0,0,0.995298,0.959215,0.976923,662.000000,ale_1_1,1811.781599
1,1,0.960926,0.995502,0.977909,667.000000,ale_1_1,1811.781599
2,accuracy,0.977427,0.977427,0.977427,0.977427,ale_1_1,1811.781599
3,macro avg,0.978112,0.977358,0.977416,1329.000000,ale_1_1,1811.781599
4,weighted avg,0.978047,0.977427,0.977418,1329.000000,ale_1_1,1811.781599
0,0,0.973887,0.993943,0.983813,3302.000000,ale_5_1,5393.620386
1,1,0.967585,0.871533,0.917051,685.000000,ale_5_1,5393.620386
2,accuracy,0.972912,0.972912,0.972912,0.972912,ale_5_1,5393.620386
3,macro avg,0.970736,0.932738,0.950432,3987.000000,ale_5_1,5393.620386
4,weighted avg,0.972804,0.972912,0.972343,3987.000000,ale_5_1,5393.620386


### Salvando

Salvando todos os resultados em um único arquivo

In [14]:
df_resultados.to_csv(f'Dados/Resultados/BERT/relatório_completo.csv', index = False)

# Salvando os Resultados por Categoria

## Funções

In [32]:
def salvar_por_categoria(df_r, categoria, nome):

    df_r[df_r["categoria"] == categoria].to_csv(f'Dados/Resultados/BERT/Resultado/{categoria}/{nome}_y.csv', index = False)


def relatorio_por_categoria(df_r, categoria, nome):

    y_teste = df_r[df_r["categoria"] == categoria]['match']
    y_pred = df_r[df_r["categoria"] == categoria]['pred']

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome

    df_relatorio.to_csv(f'Dados/Resultados/BERT/Relatório/{categoria}/{nome}_relatório.csv', index = True)

## Leitura e Salvamento

In [33]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
lista_df_resultados = []
for nome in arquivos:

    df_r = pd.read_csv(f"Dados/Resultados/BERT/Resultado/{nome}_y.csv")
    lista_df_resultados.append(df_r)

In [34]:
lista_categorias = ("celulares", "notebooks", "geladeiras", "fogoes", "tvs")
for nome, df_r in zip(arquivos, lista_df_resultados):

    for categoria in lista_categorias:

        salvar_por_categoria(df_r, categoria, nome)
        relatorio_por_categoria(df_r, categoria, nome)

c:\Users\llvs2\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llvs2\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llvs2\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llvs2\anac